In [1]:
# Standard
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import warnings; warnings.simplefilter('ignore')

# Machine Learning
from scipy import stats
import sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, precision_score, precision_recall_curve, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# NLP 
import string
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
import nltk
# nltk.download('wordnet')
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
df1 = pd.read_csv("/Users/Adam/Desktop/NLP/earning_calls.csv")
df1.head(n=10)

,href,Transcript,Ticker
0,/earnings/call-transcripts/2020/04/30/microsof...,"[""OperatorGreetings and welcome to the Microso...",msft
1,/earnings/call-transcripts/2020/01/30/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft
2,/earnings/call-transcripts/2019/10/23/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft
3,/earnings/call-transcripts/2019/07/18/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft
4,/earnings/call-transcripts/2019/04/25/microsof...,"['Prepared Remarks:', 'Operator', 'Greetings, ...",msft
5,/earnings/call-transcripts/2019/01/31/microsof...,"['Contents:', '', 'Prepared Remarks', 'Questio...",msft
6,/earnings/call-transcripts/2018/10/24/microsof...,"['Prepared Remarks:', 'Operator', 'Greetings a...",msft
7,/earnings/call-transcripts/2018/07/19/microsof...,"['Prepared Remarks:', 'Operator', 'Greetings, ...",msft
8,/earnings/call-transcripts/2018/04/26/microsof...,"['Prepared Remarks:', 'Operator', 'Greetings, ...",msft
9,/earnings/call-transcripts/2018/01/31/microsof...,"['Prepared Remarks:', 'Operator', 'Welcome to ...",msft


In [3]:
# Set index
df1['Index'] = range(1, len(df1)+1)
df1.set_index('Index')

,href,Transcript,Ticker
Index,,,
1,/earnings/call-transcripts/2020/04/30/microsof...,"[""OperatorGreetings and welcome to the Microso...",msft
2,/earnings/call-transcripts/2020/01/30/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft
3,/earnings/call-transcripts/2019/10/23/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft
4,/earnings/call-transcripts/2019/07/18/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft
5,/earnings/call-transcripts/2019/04/25/microsof...,"['Prepared Remarks:', 'Operator', 'Greetings, ...",msft
...,...,...,...
3491,/earnings/call-transcripts/2019/08/28/coty-inc...,"[""OperatorGood morning, ladies and gentlemen. ...",coty
3492,/earnings/call-transcripts/2019/05/08/coty-inc...,[],coty
3493,/earnings/call-transcripts/2019/02/08/coty-inc...,"['Contents:', '', 'Prepared Remarks', 'Questio...",coty


In [4]:
# Get Date
# Split href
href = df1['href'].str.split("/",n=6,expand=True)
# Drop unnecessary axes
href1 = href.drop([0,1,2,6], axis = 1)
# Rename Axes
href1 = href1.rename(columns = {3:"Year",4:"Month",5:"Day"})
# Create Index to merge on
href1['Index'] = range(1, len(href1)+1)
href1.set_index('Index')
href1.head()
# Join to Original Dataframe 
df1 = df1.merge(href1, how = 'outer', on = 'Index' )
# Fix the year column due to inconsistent href links
df1['Year'] = df1.href.apply(lambda x: (re.findall(r"20[0-9]{2}", x)[0]))
df1['Date'] = pd.to_datetime(df1[['Year', 'Month', 'Day']])
df1.head()

,href,Transcript,Ticker,Index,Year,Month,Day,Date
0,/earnings/call-transcripts/2020/04/30/microsof...,"[""OperatorGreetings and welcome to the Microso...",msft,1,2020,04,30,2020-04-30
1,/earnings/call-transcripts/2020/01/30/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,2,2020,01,30,2020-01-30
2,/earnings/call-transcripts/2019/10/23/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,3,2019,10,23,2019-10-23
3,/earnings/call-transcripts/2019/07/18/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,4,2019,07,18,2019-07-18
4,/earnings/call-transcripts/2019/04/25/microsof...,"['Prepared Remarks:', 'Operator', 'Greetings, ...",msft,5,2019,04,25,2019-04-25


In [5]:
#Create column with date 1 week later for comparison of stock prices
df1['Date_1Week_Before'] = df1['Date'] + pd.DateOffset(days = -7)
df1['Date_1Week_After'] = df1['Date'] + pd.DateOffset(days = 7)

In [6]:
# Get Quarter

# Split existing href
href2 = href[6].str.split(r'-\d',n = 3, expand = True)
href3 = href2[0].str.split('-q', n = 4, expand = True)

# Rename and drop unnessecary columns
href3 = href3.rename(columns = {1:'Quarter'})
href3 = href3.drop(columns = [0,2])

# Create Index
href3['Index'] = range(1, len(href1)+1)
href3.set_index('Index')
href3.head()

# Merge to existing dataframe
df1 = df1.merge(href3, how = 'outer', on = 'Index')
df1.head()

,href,Transcript,Ticker,Index,Year,Month,Day,Date,Date_1Week_Before,Date_1Week_After,Quarter
0,/earnings/call-transcripts/2020/04/30/microsof...,"[""OperatorGreetings and welcome to the Microso...",msft,1,2020,04,30,2020-04-30,2020-04-23,2020-05-07,3
1,/earnings/call-transcripts/2020/01/30/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,2,2020,01,30,2020-01-30,2020-01-23,2020-02-06,2
2,/earnings/call-transcripts/2019/10/23/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,3,2019,10,23,2019-10-23,2019-10-16,2019-10-30,1
3,/earnings/call-transcripts/2019/07/18/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,4,2019,07,18,2019-07-18,2019-07-11,2019-07-25,4
4,/earnings/call-transcripts/2019/04/25/microsof...,"['Prepared Remarks:', 'Operator', 'Greetings, ...",msft,5,2019,04,25,2019-04-25,2019-04-18,2019-05-02,3


In [7]:
#bootleg company name column
df1['CompanyName'] = df1['href'].str[38:]

In [8]:
df1.head()

,href,Transcript,Ticker,Index,Year,Month,Day,Date,Date_1Week_Before,Date_1Week_After,Quarter,CompanyName
0,/earnings/call-transcripts/2020/04/30/microsof...,"[""OperatorGreetings and welcome to the Microso...",msft,1,2020,04,30,2020-04-30,2020-04-23,2020-05-07,3,microsoft-corp-msft-q3-2020-earnings-call-tran...
1,/earnings/call-transcripts/2020/01/30/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,2,2020,01,30,2020-01-30,2020-01-23,2020-02-06,2,microsoft-corp-msft-q2-2020-earnings-call-tran...
2,/earnings/call-transcripts/2019/10/23/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,3,2019,10,23,2019-10-23,2019-10-16,2019-10-30,1,microsoft-corp-msft-q1-2020-earnings-call-tran...
3,/earnings/call-transcripts/2019/07/18/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,4,2019,07,18,2019-07-18,2019-07-11,2019-07-25,4,microsoft-corp-msft-q4-2019-earnings-call-tran...
4,/earnings/call-transcripts/2019/04/25/microsof...,"['Prepared Remarks:', 'Operator', 'Greetings, ...",msft,5,2019,04,25,2019-04-25,2019-04-18,2019-05-02,3,microsoft-corp-msft-q3-2019-earnings-call-tran...


# Ryan's Sandbox

In [9]:
# Contraction dictionary
contractions_dict = {
    "didn't": 'did not',
    "don't": 'do not',
    "aren't": 'are not',
    "can't": 'cannot',
    "could've": "could've",
    "couldn't": "could not",
    "i'll": "i will",
    "i'd": "i would",
    "i'm": "i am",
    "it'll": "it will",
    "we'll": "we will"
    
}

contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

In [10]:
# Standard function to clean string
def clean(string):
    x = string.lower() # lowercases the string
    x = expand_contractions(x) # replaces contractions
    x = re.sub(r'\W+', ' ', x) # takes only alpha numeric
    return x

In [11]:
stonks = pd.read_csv("/Users/Adam/Desktop/NLP/Company_Stock_Price_Clean.csv")

In [12]:
stonks.head(n=10)

,Date,Ticker,Adj_Close
0,2017-01-03,MSFT,58.969059
1,2017-01-03,AAPL,110.392334
2,2017-01-03,AMZN,753.669983
3,2017-01-03,FB,116.860001
4,2017-01-03,GOOGL,808.010010
5,2017-01-03,GOOG,786.140015
6,2017-01-03,JNJ,105.502319
7,2017-01-03,V,77.675957
8,2017-01-03,PG,75.750793
9,2017-01-03,JPM,79.052376


In [13]:
stonks['Ticker'] = stonks['Ticker'].astype(str)
stonks['Ticker'] = stonks['Ticker'].str.lower()

In [14]:
df1['Date'] = pd.to_datetime(df1['Date'])
stonks['Date'] = pd.to_datetime(stonks['Date'])

In [15]:
#try to merge the 2 datasets
# but this needs the Ticker on the original dataset
df_final = pd.merge(df1, stonks, on = ['Date', 'Ticker'], how = 'inner')

In [16]:
df_final = pd.merge(df_final, stonks, left_on = ['Date_1Week_Before', 'Ticker'],
                                             right_on = ['Date', 'Ticker'], how = 'inner')

In [17]:
df_final = pd.merge(df_final, stonks, left_on = ['Date_1Week_After', 'Ticker'],
                                             right_on = ['Date', 'Ticker'], how = 'inner')

In [18]:
df_final.head()

,href,Transcript,Ticker,Index,Year,Month,Day,Date_x,Date_1Week_Before,Date_1Week_After,Quarter,CompanyName,Adj_Close_x,Date_y,Adj_Close_y,Date,Adj_Close
0,/earnings/call-transcripts/2020/04/30/microsof...,"[""OperatorGreetings and welcome to the Microso...",msft,1,2020,04,30,2020-04-30,2020-04-23,2020-05-07,3,microsoft-corp-msft-q3-2020-earnings-call-tran...,178.712280,2020-04-23,170.943909,2020-05-07,183.090088
1,/earnings/call-transcripts/2020/01/30/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,2,2020,01,30,2020-01-30,2020-01-23,2020-02-06,2,microsoft-corp-msft-q2-2020-earnings-call-tran...,171.830795,2020-01-23,165.804092,2020-02-06,182.621201
2,/earnings/call-transcripts/2019/10/23/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,3,2019,10,23,2019-10-23,2019-10-16,2019-10-30,1,microsoft-corp-msft-q1-2020-earnings-call-tran...,136.023209,2019-10-16,139.165085,2019-10-30,143.327850
3,/earnings/call-transcripts/2019/07/18/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,4,2019,07,18,2019-07-18,2019-07-11,2019-07-25,4,microsoft-corp-msft-q4-2019-earnings-call-tran...,134.761719,2019-07-11,136.717636,2019-07-25,138.485886
4,/earnings/call-transcripts/2019/04/25/microsof...,"['Prepared Remarks:', 'Operator', 'Greetings, ...",msft,5,2019,04,25,2019-04-25,2019-04-18,2019-05-02,3,microsoft-corp-msft-q3-2019-earnings-call-tran...,127.109573,2019-04-18,121.420891,2019-05-02,124.216019


In [19]:
df_final = df_final.drop(columns = ['Date_y', 'Date'])

In [20]:
df_final = df_final.rename(columns = {'Adj_Close_y':'Price_Before', 'Adj_Close':'Price_After', 'Adj_Close_x': 'Adj_Close',
                               'Date_x':'Date'})

In [21]:
df_final.head(n=5)

,href,Transcript,Ticker,Index,Year,Month,Day,Date,Date_1Week_Before,Date_1Week_After,Quarter,CompanyName,Adj_Close,Price_Before,Price_After
0,/earnings/call-transcripts/2020/04/30/microsof...,"[""OperatorGreetings and welcome to the Microso...",msft,1,2020,04,30,2020-04-30,2020-04-23,2020-05-07,3,microsoft-corp-msft-q3-2020-earnings-call-tran...,178.712280,170.943909,183.090088
1,/earnings/call-transcripts/2020/01/30/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,2,2020,01,30,2020-01-30,2020-01-23,2020-02-06,2,microsoft-corp-msft-q2-2020-earnings-call-tran...,171.830795,165.804092,182.621201
2,/earnings/call-transcripts/2019/10/23/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,3,2019,10,23,2019-10-23,2019-10-16,2019-10-30,1,microsoft-corp-msft-q1-2020-earnings-call-tran...,136.023209,139.165085,143.327850
3,/earnings/call-transcripts/2019/07/18/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,4,2019,07,18,2019-07-18,2019-07-11,2019-07-25,4,microsoft-corp-msft-q4-2019-earnings-call-tran...,134.761719,136.717636,138.485886
4,/earnings/call-transcripts/2019/04/25/microsof...,"['Prepared Remarks:', 'Operator', 'Greetings, ...",msft,5,2019,04,25,2019-04-25,2019-04-18,2019-05-02,3,microsoft-corp-msft-q3-2019-earnings-call-tran...,127.109573,121.420891,124.216019


In [22]:
df_final['clean_transcript'] = df_final.Transcript.apply(lambda x: clean(x))

In [23]:
# List of stopwords
stop = list(STOPWORDS)

# Functions
# removes stop words from a clean transcript
def remove_stop(string):
    wostop = [] # empty list to append to
    word = string.split() # splits string into list
    for n, i in enumerate(word): # iterates over the list
        if i not in stop: # if word in the list not in stop words list
            wostop.append(word[n]) # append to wostop list
    wostop = ' '.join(wostop) # join wostop list into a string
    return wostop # returns a string

# Stems word
def stem(string):
    t = [] # empty list to append
    ps = nltk.stem.PorterStemmer() # stem purposes
    word = string.split() # split string into list
    for item in word: # iterate through list
        stem = ps.stem(item) # stem each word
        t.append(stem) # add new stem word into list
    t = ' '.join(t) # join list of stem words into a string
    return t # returns a string

In [24]:
df_final['clean_transcript2'] = df_final.clean_transcript.apply(lambda x: remove_stop(x))
df_final.head()

,href,Transcript,Ticker,Index,Year,Month,Day,Date,Date_1Week_Before,Date_1Week_After,Quarter,CompanyName,Adj_Close,Price_Before,Price_After,clean_transcript,clean_transcript2
0,/earnings/call-transcripts/2020/04/30/microsof...,"[""OperatorGreetings and welcome to the Microso...",msft,1,2020,04,30,2020-04-30,2020-04-23,2020-05-07,3,microsoft-corp-msft-q3-2020-earnings-call-tran...,178.712280,170.943909,183.090088,operatorgreetings and welcome to the microsof...,operatorgreetings welcome microsoft fiscal yea...
1,/earnings/call-transcripts/2020/01/30/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,2,2020,01,30,2020-01-30,2020-01-23,2020-02-06,2,microsoft-corp-msft-q2-2020-earnings-call-tran...,171.830795,165.804092,182.621201,operatorwelcome to the microsoft fiscal year ...,operatorwelcome microsoft fiscal year 2020 sec...
2,/earnings/call-transcripts/2019/10/23/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,3,2019,10,23,2019-10-23,2019-10-16,2019-10-30,1,microsoft-corp-msft-q1-2020-earnings-call-tran...,136.023209,139.165085,143.327850,operatorwelcome to the microsoft fiscal year ...,operatorwelcome microsoft fiscal year 2020 fir...
3,/earnings/call-transcripts/2019/07/18/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,4,2019,07,18,2019-07-18,2019-07-11,2019-07-25,4,microsoft-corp-msft-q4-2019-earnings-call-tran...,134.761719,136.717636,138.485886,operatorwelcome to the microsoft fiscal year ...,operatorwelcome microsoft fiscal year 2019 fou...
4,/earnings/call-transcripts/2019/04/25/microsof...,"['Prepared Remarks:', 'Operator', 'Greetings, ...",msft,5,2019,04,25,2019-04-25,2019-04-18,2019-05-02,3,microsoft-corp-msft-q3-2019-earnings-call-tran...,127.109573,121.420891,124.216019,prepared remarks operator greetings welcome t...,prepared remarks operator greetings welcome mi...


In [25]:
df_final = df_final.drop(columns = ['Index'])
df_final = df_final.drop(columns = ['clean_transcript'])

In [26]:
df_final = (df_final[df_final.Transcript.apply(lambda x: len(x) > 10)])
df_final

,href,Transcript,Ticker,Year,Month,Day,Date,Date_1Week_Before,Date_1Week_After,Quarter,CompanyName,Adj_Close,Price_Before,Price_After,clean_transcript2
0,/earnings/call-transcripts/2020/04/30/microsof...,"[""OperatorGreetings and welcome to the Microso...",msft,2020,04,30,2020-04-30,2020-04-23,2020-05-07,3,microsoft-corp-msft-q3-2020-earnings-call-tran...,178.712280,170.943909,183.090088,operatorgreetings welcome microsoft fiscal yea...
1,/earnings/call-transcripts/2020/01/30/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,2020,01,30,2020-01-30,2020-01-23,2020-02-06,2,microsoft-corp-msft-q2-2020-earnings-call-tran...,171.830795,165.804092,182.621201,operatorwelcome microsoft fiscal year 2020 sec...
2,/earnings/call-transcripts/2019/10/23/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,2019,10,23,2019-10-23,2019-10-16,2019-10-30,1,microsoft-corp-msft-q1-2020-earnings-call-tran...,136.023209,139.165085,143.327850,operatorwelcome microsoft fiscal year 2020 fir...
3,/earnings/call-transcripts/2019/07/18/microsof...,"[""OperatorWelcome to the Microsoft Fiscal Year...",msft,2019,07,18,2019-07-18,2019-07-11,2019-07-25,4,microsoft-corp-msft-q4-2019-earnings-call-tran...,134.761719,136.717636,138.485886,operatorwelcome microsoft fiscal year 2019 fou...
4,/earnings/call-transcripts/2019/04/25/microsof...,"['Prepared Remarks:', 'Operator', 'Greetings, ...",msft,2019,04,25,2019-04-25,2019-04-18,2019-05-02,3,microsoft-corp-msft-q3-2019-earnings-call-tran...,127.109573,121.420891,124.216019,prepared remarks operator greetings welcome mi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3227,/earnings/call-transcripts/2019/11/06/coty-inc...,"[""OperatorGood morning, ladies and gentlemen. ...",coty,2019,11,06,2019-11-06,2019-10-30,2019-11-13,1,coty-inc-coty-q1-2020-earnings-call-transcript,12.747121,11.386252,11.983468,operatorgood morning ladies gentlemen name mar...
3228,/earnings/call-transcripts/2019/08/28/coty-inc...,"[""OperatorGood morning, ladies and gentlemen. ...",coty,2019,08,28,2019-08-28,2019-08-21,2019-09-04,4,coty-inc-coty-q4-2019-earnings-call-transcript,9.019238,8.555227,9.077241,operatorgood morning ladies gentlemen name mar...
3230,/earnings/call-transcripts/2019/02/08/coty-inc...,"['Contents:', '', 'Prepared Remarks', 'Questio...",coty,2019,02,08,2019-02-08,2019-02-01,2019-02-15,2,coty-inc-coty-q2-2019-earnings-conference-call...,8.832841,7.138223,10.451722,contents prepared remarks questions answers ca...
3231,/earnings/call-transcripts/2018/11/07/coty-inc...,"['Prepared Remarks:', 'Operator', ""Good mornin...",coty,2018,11,07,2018-11-07,2018-10-31,2018-11-14,1,coty-inc-coty-q1-2019-earnings-conference-call...,8.078259,9.841298,8.115572,prepared remarks operator good morning ladies ...


In [27]:
#df_final.to_csv('Transcripts_Clean.csv')